# Sesión 13: Regresión Logística con Python

### Fuga (churn) de clientes con Regresión Logística
Una compañía de telecomunicaciones está preocupada por la cantidad de clientes que abandonan su negocio de telefonía fija por competidores de cable. Necesitan entender quién se va. Imagina que eres analista de esta empresa y tienes que averiguar quién se va y por qué.

Vamos a importar las librerías requeridas:

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

<h2 id="about_dataset">Sobre el archivo de datos</h2>
Utilizaremos un conjunto de datos de telecomunicaciones para predecir la pérdida de clientes. Este es un conjunto de datos de clientes históricos donde cada fila representa un cliente. Los datos son relativamente fáciles de entender y puedes descubrir información que puedes usar de inmediato. Por lo general, es menos costoso mantener clientes que adquirir nuevos, por lo que el enfoque de este análisis es predecir los clientes que se quedarán con la empresa.


Este conjunto de datos proporciona información para ayudarte a predecir qué comportamiento te ayudará a retener clientes. Puedes analizar todos los datos relevantes del cliente y desarrollar programas enfocados en la retención de clientes.



El conjunto de datos incluye información sobre:
- Clientes que se fueron en el último mes: la columna se llama Churn.
- Servicios a los que se ha suscrito cada cliente: teléfono, líneas múltiples, Internet, seguridad en línea, respaldo en línea, protección del dispositivo, soporte técnico y transmisión de TV y películas.
- Información de la cuenta del cliente: cuánto tiempo han sido clientes, contrato, método de pago, facturación electrónica, cargos mensuales y cargos totales.
- Información demográfica sobre los clientes: género, rango de edad y si tienen socios y dependientes.

###  Carga los datos de Telco Churn
Telco Churn es un archivo de datos hipotéticos que se refiere a los esfuerzos de una empresa de telecomunicaciones para reducir la rotación en su base de clientes. Cada caso corresponde a un cliente separado y registra diversa información demográfica y de uso del servicio. Antes de poder trabajar con los datos, debes usar la URL para obtener ChurnData.csv.

### Carga la data del archivo CSV

In [ ]:
churn_df = pd.read_csv("data/ChurnData.csv")
churn_df.head(10)

<h2 id="preprocessing">Pre-procesamiento y selección de la data</h2>

Vamos a seleccionar algunas características para el modelado. También, cambiar el tipo de datos de destino para que sea entero, ya que es un requisito del algoritmo scikitlearn:

In [ ]:
churn_df = churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']]
churn_df['churn'] = churn_df['churn'].astype('int')
churn_df.head()

In [ ]:
churn_df.shape

Vamos a definir <b>X</b>, and <b>y</b> para nuestro conjunto de datos:

In [ ]:
X = np.asarray(churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
X[0:5]

In [ ]:
y = np.asarray(churn_df['churn'])
y [0:5]

Además, nosotros normalizamos el conjunto de datos:

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Conjunto de datos Train/Test

Bien, dividimos nuestro conjunto de datos en un conjunto de entrenamiento y de prueba:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

<h2 id="modeling">Modelando (Regresión Logística con Scikit-learn)</h2>

Vamos a construir nuestro modelo usando __LogisticRegression__ del paquete Scikit-learn. Esta función implementa la regresión logística y puedes usar diferentes optimizadores numéricos para encontrar parámetros, incluidos los solucionadores 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'. Puedes buscar información extensa sobre las ventajas y desventajas de estos optimizadores si los buscas en Internet.
<br>La versión de Regresión logística en Scikit-learn, admite la regularización. La regularización es una técnica utilizada para resolver el problema de sobreajuste en los modelos de aprendizaje automático.</br>
<br> El parámetro **C** indica el __inverso de la intensidad de regularización__, que debe ser un float positivo. Los valores más pequeños especifican una regularización más fuerte.</br>
Ahora ajustemos nuestro modelo con el conjunto de datos de entrenamiento:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

In [ ]:
#vemos los componentes del modelo de regresión obtenido
dir(LR)

Con el siguiente comando mostramos los coeficientes recultantes del análisis de regresión:

In [ ]:
print(['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip'])
LR.coef_

Ahora nosotros podemos predecir con nuestro data de prueba:

In [ ]:
yhat = LR.predict(X_test)
yhat

__predict_proba__  devuelve estimaciones para todas las clases, ordenadas por la etiqueta de clases. Entonces, la primera columna es la probabilidad de la clase 1, P(Y=1|X), y la segunda columna es la probabilidad de la clase 0, P(Y=0|X):

In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

<h2 id="evaluation">Evaluación</h2>

### Índice de jaccard
Probemos el índice jaccard para la evaluación de precisión. Podemos definir jaccard como el tamaño de la intersección dividido por el tamaño de la unión de dos conjuntos de etiquetas. Si el conjunto completo de etiquetas pronosticadas para una muestra coincide estrictamente con el conjunto real de etiquetas, la precisión del subconjunto es 1.0; de lo contrario es 0.0.


In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, yhat)

### Matriz de confusión
Otra forma de ver la precisión del clasificador es mirar la __matriz de confusión__.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],normalize= False,  title='Confusion matrix')

Mira la primera fila. La primera fila es para clientes cuyo valor de churn real en el conjunto de prueba es 1.
Como puedes calcular, de 40 clientes, el valor de churn de 15 de ellos es 1.
Y de estos 15, el clasificador predijo correctamente 6 de ellos como 1 y 9 de ellos como 0.

Significa que, para 6 clientes, el valor de churn real era 1 en el conjunto de prueba, y el clasificador también predijo correctamente esos como 1. Sin embargo, mientras que la etiqueta real de 9 clientes era 1, el clasificador predijo aquellos como 0, lo que no es muy bueno. Podemos considerarlo como un error del modelo para la primera fila.

¿Qué pasa con los clientes con valor 0 de churn? Veamos la segunda fila.
Parece que había 25 clientes cuyo valor de churn era 0.


El clasificador pronosticó correctamente 24 de ellos como 0, y uno de ellos erróneamente como 1. Por lo tanto, ha hecho un buen trabajo al predecir a los clientes con un valor 0 de churn. Lo bueno de la matriz de confusión es que muestra la capacidad del modelo para predecir correctamente o separar las clases. En el caso específico del clasificador binario, como este ejemplo, podemos interpretar estos números como el recuento de verdaderos positivos, falsos positivos, verdaderos negativos y falsos negativos.

In [ ]:
print (classification_report(y_test, yhat))


En función del recuento de cada sección, podemos calcular la precisión y la recuperación de cada etiqueta:


- __Precision__ es una medida de la precisión siempre que se haya predicho una etiqueta de clase. Se define por: precisión = TP / (TP + FP)

- __Recall__ Es la verdadera tasa positiva. Se define como: Recall =  TP / (TP + FN)

    
Entonces, podemos calcular precision y recall para cada clase.

__F1 score:__
Ahora estamos en condiciones de calcular los puntajes F1 para cada etiqueta en función de la precisión y la recuperación de esa etiqueta.

El puntaje F1 es el promedio armónico de la precisión y la recuperación, donde un puntaje F1 alcanza su mejor valor en 1 (precisión perfecta y recuperación) y el peor en 0. Es una buena manera de mostrar que un clasificador tiene un buen valor para ambos precision y recall.


Y finalmente, podemos decir que la precisión promedio para este clasificador es el promedio del puntaje F1 para ambas etiquetas, que es 0,72 en nuestro caso.

In [ ]:
from sklearn import metrics
print("Accuracy Score: ", metrics.accuracy_score(y_test, yhat))
print("Confusion Matrix:\n ", metrics.confusion_matrix(y_test, yhat))
print("Precision Score: ", metrics.precision_score(y_test, yhat))
print("Recall Score: ", metrics.recall_score(y_test, yhat))
print("F1 Score: ", metrics.f1_score(y_test, yhat))

### Log loss
Ahora, intentemos __log loss__ para la evaluación. En la regresión logística, el resultado puede ser la probabilidad de que el abandono del cliente sea sí (o igual a 1). Esta probabilidad es un valor entre 0 y 1.
La __log loss__ (pérdida logarítmica) mide el rendimiento de un clasificador donde la salida predicha es un valor de probabilidad entre 0 y 1.

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

### AUC (curva ROC)

Una curva ROC (curva de característica operativa del receptor) es un gráfico que muestra el rendimiento de un modelo de clasificación en todos los umbrales de clasificación. Esta curva representa dos parámetros:

* Tasa de verdaderos positivos (TPR)
* Tasa de falsos positivos (FPR)

Una curva ROC representa TPR frente a FPR en diferentes umbrales de clasificación. Reducir el umbral de clasificación clasifica más elementos como positivos, por lo que aumentarán tanto los falsos positivos como los verdaderos positivos. Como la curva ROC es una función, para interpretarla se emplea el área bajo la curva (AUC). AUC representa la probabilidad de que una muestra positiva se sitúe con un score superior al de una muestra negativa. Podemos llamarlo su poder de discriminación.

A modo de guía para interpretar las curvas ROC se han establecido los siguientes intervalos para los valores de AUC:

* [0.5]: Es como lanzar una moneda.
* [0.5, 0.6): rendimiento malo.
* [0.6, 0.75): rendimiento regular.
* [0.75, 0.9): rendimiento bueno.
* [0.9, 0.97): rendimiento muy bueno.
* [0.97, 1): rendimiento excelente.

In [ ]:
yhat

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(y_test, yhat)
auc(fpr, tpr)

### Ejecutando con varios optimizadores y reguladores
Construiremos el modelo de Regresión logística nuevamente para el mismo conjunto de datos, pero esta vez, usa diferentes valores de <b>solver</b> y <b>regularization</b>. Con esto podemos hacer comparaciones de varios modelos hasta encontrar el más adecuado:

In [ ]:
solvers=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
Cs=[0.01,0.02,0.05,0.1]

for s in solvers:
    for c in Cs:
        LR = LogisticRegression(C=c, solver=s).fit(X_train,y_train)
        yhat = LR.predict(X_test)
        yhat_prob = LR.predict_proba(X_test)
        log_loss(y_test, yhat_prob)
        fpr, tpr, threshold = roc_curve(y_test, yhat)
        print("Solver="+s+", C="+str(c)+
              "->Accuracy: "+str(metrics.accuracy_score(y_test, yhat)) + 
             "->Log Loss : "+str(log_loss(y_test, yhat_prob)) +
             "->AUC : "+str(auc(fpr, tpr)))
